# 案例：树洞软件中的NLP应用
进入大三以来，压力比较大，因而接触到了一款树洞软件——“可话”，在这个软件的平台上我可以比较自在地分享一些事情。同时，“可话”的一大亮点是可以根据你的内容找到与你“共鸣”的话，你可以选择为其点赞和评论，从而交到志同道合的朋友。

分析这款软件的背后逻辑，我们认为他有两大核心要点：
1. 将“共鸣”这一应用抽象为**文本匹配**任务
2. 通过用户行为可以提供标注数据，便于后续的模型调整

## 1. 处理原始数据
将数据集分为训练和测试的data，匹配的词典all_sents

In [1]:
import numpy as np
import pandas as pd

In [2]:
path = r'./dataset/'

data = {"sent1":[],"sent2":[],"label":[]}
with open(path+"simtrain_to05sts.txt",'r',encoding='utf-8') as f:
    for line in f:
        s = line.replace('\n','').split('\t')
        sent1,sent2,label = s[1],s[3],eval(s[4])
        data["sent1"].append(sent1)
        data["sent2"].append(sent2)
        data["label"].append(label)
data_df = pd.DataFrame(data,index=None)
data_df

,sent1,sent2,label
0,咱俩谁跟谁呀。,我们俩谁跟谁呀。,5.0
1,咱俩谁跟谁呀。,咱哥俩谁跟谁呀。,5.0
2,我拿了汪老师一本书。,我拿了汪老师的一本书。,5.0
3,我给了她一只笔。,我拿了一支笔给她。,4.0
4,咱俩谁跟谁呀。,咱俩关系很好。,4.0
...,...,...,...
12742,他没努力去解决这个问题，这个问题对他本来就很容易。,港大法律学院助理教授张达明也指，五区公投运动无涉及任何违反基本法的问题，只是一个政治行动。,0.0
12743,他没努力去解决这个问题，这个问题对他本来就很容易。,这个方法以前曾暂时用于工业生产，但因价格昂贵而停止使用。,0.0
12744,他没努力去解决这个问题，这个问题对他本来就很容易。,豆腐如果没有妥善保存或冰箱冷度不够，很容易馊掉，可以闻到或吃到酸味，此时不可再食用。,0.0
12745,他没努力去解决这个问题，这个问题对他本来就很容易。,事实上，并不是每一具PSO-1瞄准镜都一定具有这个功能，目前在市场上有很多外型与PSO-1瞄...,0.0


In [3]:
data_df.to_csv(path+"data.csv",encoding='utf_8_sig',index=None)

In [4]:
all_sents = {"sents":[]}
with open(path+"simtrain_to05sts.txt",'r',encoding='utf-8') as f:
    for line in f:
        s = line.replace('\n','').split('\t')
        sent = s[1]
        all_sents["sents"].append(sent)
all_sents['sents'] = list(set(all_sents['sents']))
all_sents_df = pd.DataFrame(all_sents,index=None)
all_sents_df

,sents
0,这位老板娘我很早就认识她了。
1,这个人办起事来，别提多啰嗦了。
2,我习惯喝咖啡不放糖。
3,事情他已经去办了，成不成还不知道。
4,因为这件事，小田受到了表扬。
...,...
995,他们家常常出去旅游。
996,对方的后卫把球踢进了自家的球门。
997,卫星定位信号经常被干扰。
998,关山月，广东阳江人。


In [5]:
all_sents_df.to_csv(path+"all_sents.csv",encoding='utf_8_sig',index=None)

## 2. 分析数据集
句子长度最长为196，最短为3，平均为24.5.我们选择150作为最大长度，可以看到有99.8的句子比这个小，所以是比较合适的

In [6]:
length_list = []
for sent in list(data_df['sent1'])+list(data_df['sent2']):
    length_list.append(len(sent))
print("max = {}, min = {}, mean = {}".format(np.max(length_list),np.min(length_list),np.mean(length_list)))

max = 196, min = 3, mean = 24.523260374990194


In [7]:
# 根据上述结果，我们尝试取150
print(sum(1 if _ <= 150 else 0 for _ in length_list)/len(length_list))

0.9978426296383462


In [8]:
sent_max_len = 150

## 3. 将句子转换为字级别的词向量
转换为字级别而没有转换为词组级别原因如下：
字向量受到分词效果的影响较小，且embedding层的维数远远低于词向量

In [9]:
df = pd.read_csv(path+"data.csv")

# 标签及词汇表
labels, evaluations = list(df['label'].unique()), list(df['sent1'].unique())+list(df['sent2'].unique())

# 构造词组级别的特征
vocabulary = []
for evaluation in evaluations:
    for word in evaluation:
        vocabulary.append(word)

vocabulary = set(vocabulary)

# 字典列表
word_dictionary = {word: i+1 for i, word in enumerate(vocabulary)}
inverse_word_dictionary = {i+1: word for i, word in enumerate(vocabulary)}
label_dictionary = {label: i for i, label in enumerate(labels)}
output_dictionary = {i: labels for i, labels in enumerate(labels)}

vocab_size = len(word_dictionary.keys()) # 词汇表大小
label_size = len(label_dictionary.keys()) # 标签类别数量

# 序列填充，按input_shape填充，长度不足的按0补充
x1 = [[word_dictionary[word] for word in sent] for sent in df['sent1']]
for i in range(len(x1)):
    if len(x1[i])>sent_max_len:
        x1[i] = x1[i][:sent_max_len]
    else:
        x1[i] = x1[i] + list(0 for j in range(sent_max_len-len(x1[i])))
x2 = [[word_dictionary[word] for word in sent] for sent in df['sent2']]
for i in range(len(x2)):
    if len(x2[i])>sent_max_len:
        x2[i] = x2[i][:sent_max_len]
    else:
        x2[i] = x2[i] + list(0 for j in range(sent_max_len-len(x2[i])))
x = [np.array([x1[i],x2[i]]) for i in range(len(x1))]
x = np.array(x)

y = [_ for _ in df['label']]
y = [np.array(label, dtype=np.float32) for label in y]
y = np.array(y)

## 4. 模型的训练
本部分使用一个简单的DSSM_LSTM进行训练

In [10]:
import torch 
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [11]:
class DSSM_LSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_size, dropout):
        super(DSSM_LSTM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_size, num_layers=1, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(150*160, 50)
        self.dropout = nn.Dropout(dropout)

    def forward(self, a, b):
        a = self.embed(a).sum(1)
        b = self.embed(b).sum(1)

        a = self.lstm(a)[0]
        b = self.lstm(b)[0]

        a = self.dropout(a).reshape(-1, 150*160)
        b = self.dropout(b).reshape(-1, 150*160)

        a = self.fc(a)
        b = self.fc(b)

        cosine = torch.cosine_similarity(a, b, dim=1, eps=1e-8)   # 计算两个句子的余弦相似度

        return cosine

In [12]:
# 使用GPU加速
use_gpu = torch.cuda.is_available()

model = DSSM_LSTM(vocab_size=vocab_size+1, embedding_dim=100, hidden_size=80, dropout=0.2).cuda()
model

DSSM_LSTM(
  (embed): Embedding(4684, 100)
  (lstm): LSTM(100, 80, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=24000, out_features=50, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [13]:
print(x)

[[[1597  555 4314 ...    0    0    0]
  [2249  615  555 ...    0    0    0]]

 [[1597  555 4314 ...    0    0    0]
  [1597 4251  555 ...    0    0    0]]

 [[2249 1521 1036 ...    0    0    0]
  [2249 1521 1036 ...    0    0    0]]

 ...

 [[1585 3589 1015 ...    0    0    0]
  [ 316 4178  954 ...    0    0    0]]

 [[1585 3589 1015 ...    0    0    0]
  [2218 3142 4653 ...    0    0    0]]

 [[1585 3589 1015 ...    0    0    0]
  [3275  181 3779 ...    0    0    0]]]


In [14]:
x = torch.from_numpy(x)
y = torch.from_numpy(y)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.1, random_state = 0)
train_x = train_x.unsqueeze(1)
test_x = test_x.unsqueeze(1)
train_x1 = train_x[:,:,0].cuda()
train_x2 = train_x[:,:,1].cuda()
test_x1 = test_x[:,:,0].cuda()
test_x2 = test_x[:,:,1].cuda()
train_y = train_y.cuda()
test_y = test_y.cuda()

tensor([[[1597,  555, 4314,  ...,    0,    0,    0],
         [2249,  615,  555,  ...,    0,    0,    0]],

        [[1597,  555, 4314,  ...,    0,    0,    0],
         [1597, 4251,  555,  ...,    0,    0,    0]],

        [[2249, 1521, 1036,  ...,    0,    0,    0],
         [2249, 1521, 1036,  ...,    0,    0,    0]],

        ...,

        [[1585, 3589, 1015,  ...,    0,    0,    0],
         [ 316, 4178,  954,  ...,    0,    0,    0]],

        [[1585, 3589, 1015,  ...,    0,    0,    0],
         [2218, 3142, 4653,  ...,    0,    0,    0]],

        [[1585, 3589, 1015,  ...,    0,    0,    0],
         [3275,  181, 3779,  ...,    0,    0,    0]]], dtype=torch.int32)


In [14]:
# 设置批大小
BATCH_SIZE = 50
# 设置损失函数
loss_function = nn.MSELoss().cuda()
# 设置优化器
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [15]:
import time
import random

In [16]:
train_N = train_x.shape[0]
max_acc = 0.0
# 训练
for epoch in range(20):
    batchindex = list(range(int(train_N / BATCH_SIZE))) # 按照批次大小，划分训练集合，得到对应批数据索引
    random.shuffle(batchindex)   # 打乱索引值，便于训练
    score_train = 0.0
    for i in batchindex:
        # 选取对应批次数据的输入和标签
        batch_x1 = train_x1[i*BATCH_SIZE: (i+1)*BATCH_SIZE]
        batch_x2 = train_x2[i*BATCH_SIZE: (i+1)*BATCH_SIZE]
        batch_y = train_y[i*BATCH_SIZE: (i+1)*BATCH_SIZE]
        # 模型预测
        y_hat = model(batch_x1, batch_x2)
        loss = loss_function(y_hat, batch_y)
        optimizer.zero_grad()   # 梯度清零
        loss.backward() # 计算梯度
        optimizer.step()    # 更新参数
        y_hat = torch.tensor([_ for _ in y_hat]).cuda()
        score_train += torch.sum(y_hat == batch_y).float() / train_y.shape[0]

    # test
    y_hat = model(test_x1, test_x2)
    y_hat = torch.tensor([_ for _ in y_hat]).cuda()
    score = torch.sum(y_hat == test_y).float() / test_y.shape[0]
    print(f"epoch: {epoch}, train loss: {loss:.4f}, train accuracy: {score_train:.4f}, test accuracy: {score:.4f}, time: {time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()) }")

    if score > max_acc:
        torch.save(model, "BEST_checkpoint.pt")

RuntimeError: CUDA out of memory. Tried to allocate 584.00 MiB (GPU 0; 4.00 GiB total capacity; 1.46 GiB already allocated; 559.50 MiB free; 1.78 GiB reserved in total by PyTorch)

## 5. 用户行为的模拟阶段
这一步实现两个任务：

（1）用户输入一个句子，模型给出最佳匹配的6个句子（和软件中的设置一致）

（2）用户根据自己的判断，选择是否要对给定的匹配“点亮”或“评论”。这里我们假定如果用户进行“点亮”或“评论”的行为，则将“label”设为“1”，不做任何操作则设为“-1”。那么，我们可以将（用户输入的句子，给定的匹配句子，label）这一三元组按顺序加载到数据集中，用做后续的模型微调（时间戳隐含于数据集的顺序中，微调时只需取出时间较近的句子）

In [1]:
# 加载模型
model = torch.load("BEST_checkpoint.pt")
model.eval()

NameError: name 'torch' is not defined

In [ ]:
# 导入词典
df = pd.read_csv(path+"all_sents.csv")
# 序列填充，按input_shape填充，长度不足的按0补充
x = [[word_dictionary[word] for word in sent] for sent in df['sents']]
for i in range(len(x)):
    if len(x[i])>sent_max_len:
        x[i] = x[i][:sent_max_len]
    else:
        x[i] = x[i] + list(0 for j in range(sent_max_len-len(x[i])))
x = [np.array(x[i]) for i in range(len(x))]
x = np.array(x)
len_x = len(x)
x = torch.from_numpy(x)
x = x.unsqueeze(1)

In [ ]:
# 用户输入句子，并将句子做one-hot编码
sent = "屋里一个人也没有。"
y = sent
y = [[word_dictionary[word] for word in y] for i in range(len_x)]
for i in range(len_x):
    if len(y[i])>sent_max_len:
        y[i] = y[i][:sent_max_len]
    else:
        y[i] = y[i] + list(0 for j in range(sent_max_len-len(y[i])))
y = np.array(y)
y = torch.from_numpy(y)
y = y.unsqueeze(1)

In [ ]:
# 进行预测
x = x.cuda()
y = y.cuda()
y_hat = list(model(x, y))

In [ ]:
counts = {i: score for i, score in enumerate(y_hat)}
items=sorted(list(counts.items()),key=lambda x:x[1],reverse=True)
# 输出6条
for i in range(6):
    sent_ = df["sents"][items[i][0]]
    print(f"No: {i}: {sent_}")

No: 0: 那个瓜没十斤重。
No: 1: 屋里连一个人也没有。
No: 2: 人人都知道这件事。
No: 3: 老雷一个字也没有透漏。
No: 4: 他这个人就知道吃。
No: 5: 一个人难免犯错误。


从结果看出，模型具有了一定的判断是否相似的能力，但是由于数据库过小（只有1000条），所以没有找到太好的匹配。这是很重要的一点问题，关系着是否能在上线后得到改进的问题（因为用户产生的新数据很可能都是负样本，样本过于不平衡的话，不利于模型的微调）

这里我们假设用户对“屋里连一个人也没有。”和“一个人难免犯错误。”进行“点亮”或“评论”，则将六条新数据插入数据集data的最后，将用户的一条数据插入数据库词典all_sents中

In [ ]:
# 插入到data
df_1 = pd.read_csv(path+"data.csv")
df_2 = pd.read_csv(path+"all_sents.csv")
sent1 = sent
for i in range(6):
    label = -1.0
    if i == 2 or i == 5:
        label = 1.0
    sent2 = df_2["sents"][items[i][0]]
    df_1.loc[len(df_1)] = [sent1, sent2, label]
df_1.to_csv(path+"data.csv", encoding='utf_8_sig',index=None)

# 插入到all_sents
df_2.loc[len(df_2)] = [sent1]
df_2.to_csv(path+"all_sents.csv", encoding='utf_8_sig',index=None)

## 6. 后续模型微调的模拟
由于没有真实场景的用户反馈，所以只介绍思路：
1. 导入保存的模型，设置为可训练的
2. 用较新的数据进行训练

## 总结
这次大作业中，选择了自己一个比较感兴趣的偏应用的nlp任务，在实践中也对树洞类软件进行了调研。未来如果有时间的话，希望自己能将其真正上线